In [21]:
import ee
import geemap
import random

In [22]:
L8bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

In [23]:
Map = geemap.Map()
# Map

In [24]:
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to TOA reflectance, without the QA bands.
    return image.updateMask(mask).divide(10000).select(L8bands).copyProperties(image, ["system:time_start"])

In [25]:
# Creating an add variable function for Landsat 8 index calculation.
def compute_indices(image):
    NDVI = image.expression('(B5-B4)/(B5+B4)', 
                            {'B4': image.select('B4'),
                             'B5': image.select('B5')}).rename('ndvi')
    MNDWI = image.expression('(B3-B6) / (B3+B6)',
                             {'B3': image.select('B3'),
                              'B6': image.select('B6')}).rename('mndwi')
    EVI = image.expression('2.5 * ((B5 - B4)/(B5 +(6*B4)-(7.5*B2)+1))',
                           {'B2': image.select('B2'),
                            'B4': image.select('B4'),
                            'B5': image.select('B5')}).rename('evi')
    LSWI = image.expression('(B5-B6)/(B5+B6)', 
                            {'B5': image.select('B5'),
                             'B6': image.select('B6')}).rename('lswi')
    return image.addBands(NDVI).addBands(MNDWI).addBands(EVI).addBands(LSWI)

In [26]:
point = ee.Geometry.Point([106.4943, 20.4858])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2020-01-01', '2020-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
# Map

In [27]:
# Add the boundary of Thai Binh province
# red_river_delta_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Provinces_Polygon/Provinces_Poly.shp'
red_river_delta_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Provinces_PolygonN/Provinces_Poly.shp'
red_river_delta = geemap.shp_to_ee(red_river_delta_shp)

In [28]:
Map.addLayer(red_river_delta, {},"TB+ND+HP provinces")
Map.centerObject(red_river_delta, 9)
Map

Map(center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['position'], widget=HBox(…

In [29]:
image_clip = image.clip(red_river_delta)
# Map.centerObject(point, 8)
# Map.addLayer(image_clip, vis_params, "Landsat-8")
# Map

In [30]:
img_compute = compute_indices(image_clip)

In [31]:
image_select = img_compute.select('ndvi', 'mndwi', 'evi', 'lswi')

In [32]:
# Add training markers from shape files
# agriculture_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Agriculture_Training.shp'
# evergreen_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Evergreen_Training.shp'
# mangrove_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Mangrove.shp'
# others_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Other.shp'
# residence_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Residental area_Training.shp'
# shrimp_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Shrimp.shp'
# water_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Markers 20201205/Training/Water_Training.shp'

agriculture_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Agriculture_Training.shp'
evergreen_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Evergreen_Training.shp'
mangrove_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Mangrove.shp'
others_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Other.shp'
residence_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Residental area_Training.shp'
shrimp_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Shrimp.shp'
water_shp = 'D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/Sample_final/Sample/Training/Water_Training.shp'

# D:\Software\Jupyter\Thesis\Learning\GEE\Thuy\Sample_final\Sample\Training

# agriculture_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/agriculture_markers.shp'
# evergreen_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/evergreen_markers.shp'
# mangrove_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/mangrove_markers.shp'
# others_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/other_markers.shp'
# residence_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/residence_markers.shp'
# shrimp_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/shrimp_markers.shp'
# water_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/water_markers.shp'
    
# Shape files to ee objects
tr_agriculture = geemap.shp_to_ee(agriculture_shp)
tr_evergreen = geemap.shp_to_ee(evergreen_shp)
tr_mangrove = geemap.shp_to_ee(mangrove_shp)
tr_others = geemap.shp_to_ee(others_shp)
tr_residence = geemap.shp_to_ee(residence_shp)
tr_shrimp = geemap.shp_to_ee(shrimp_shp)
tr_water = geemap.shp_to_ee(water_shp)

# Make markers collections
tr_classNames = tr_agriculture.merge(tr_evergreen).merge(tr_mangrove). \
                               merge(tr_others).merge(tr_residence). \
                               merge(tr_shrimp).merge(tr_water)
# Show the markers on map
Map.addLayer(tr_classNames, {}, 'Markers')
Map

Map(bottom=58182.0, center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['position…

In [33]:
# Set bands and label for classifier
bands = ['ndvi', 'mndwi', 'evi', 'lswi']
label = 'landcover'

In [34]:
sample = image_select.select(bands).sampleRegions(**{
    'collection': tr_classNames,
    'properties': [label],
    'scale': 30
})

In [35]:
# Adds a column of deterministic pseudorandom numbers. 
random.seed(2020)
sample = sample.randomColumn()
split = 0.7

In [36]:
# get training data which is 0.7 of sample
training = sample.filter(ee.Filter.lt('random', split))

In [37]:
# get validation/test data which is 0.3 of sample
validation = sample.filter(ee.Filter.gte('random', split))

In [38]:
# classify training data by randomForest
# img_classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [39]:
# Run the classification
classified = image_select.select(bands).classify(classifier)

In [40]:
# Display classification
cl_vis_params = {
    'min': 1, 
    'max': 7, 
    'palette': ['#98ff00','#0b4a8b','#ffc82d','#00ffff','#bf04c2','#ff0000','#008800']
}

legend_keys = ['Mangrove', 'Shrimp', 'Residence', 'Water','Agriculture','Other','Evergreen']
legend_colors = ['#98ff00','#0b4a8b','#ffc82d','#00ffff','#bf04c2','#ff0000','#008800']

Map.centerObject(red_river_delta, 10)
Map.addLayer(classified, cl_vis_params, 'classification')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

Map(bottom=58182.0, center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['position…

Accuracy assessment

In [45]:
# Training dataset
train_accuracy = classifier.confusionMatrix()

In [46]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 529, 0, 0, 2, 3, 0, 3],
 [0, 5, 252, 1, 1, 3, 0, 0],
 [0, 0, 0, 175, 0, 2, 1, 0],
 [0, 2, 6, 0, 70, 4, 0, 0],
 [0, 13, 1, 10, 1, 305, 0, 0],
 [0, 1, 2, 11, 0, 6, 52, 1],
 [0, 3, 0, 0, 1, 1, 0, 103]]

In [47]:
train_accuracy.accuracy().getInfo()

0.9464968152866242

In [48]:
train_accuracy.kappa().getInfo()

0.9318800245455052

In [49]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9851024208566108],
 [0.9618320610687023],
 [0.9831460674157303],
 [0.8536585365853658],
 [0.9242424242424242],
 [0.7123287671232876],
 [0.9537037037037037]]

In [50]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9566003616636528,
  0.9655172413793104,
  0.8883248730964467,
  0.9333333333333333,
  0.941358024691358,
  0.9811320754716981,
  0.9626168224299065]]

Validation dataset

In [51]:
validated = validation.classify(classifier)

In [52]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '1_1_1_1_1_1_5_0',
 'properties': {'classification': 7,
  'evi': 1.3731909028256375,
  'landcover': 5,
  'lswi': 0.04333676025271416,
  'mndwi': -0.03665893152356148,
  'ndvi': 0.07000438868999481,
  'random': 0.9222614823899629}}

In [53]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [54]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 198, 6, 1, 4, 13, 1, 4],
 [0, 5, 77, 3, 10, 7, 0, 2],
 [0, 3, 1, 50, 2, 5, 5, 0],
 [0, 4, 14, 1, 17, 7, 0, 3],
 [0, 20, 6, 11, 3, 97, 2, 3],
 [0, 3, 3, 4, 0, 14, 10, 0],
 [0, 5, 0, 1, 2, 1, 0, 28]]

In [55]:
test_accuracy.accuracy().getInfo()

0.7271341463414634

In [56]:
test_accuracy.kappa().getInfo()

0.6507400180840432

In [57]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.8722466960352423],
 [0.7403846153846154],
 [0.7575757575757576],
 [0.3695652173913043],
 [0.6830985915492958],
 [0.29411764705882354],
 [0.7567567567567568]]

In [34]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.8388888888888889,
  0.8059701492537313,
  0,
  0.5789473684210527,
  0.72,
  0,
  0.9032258064516129]]

In [35]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/'), 'Results')
#D:\Software\Jupyter\Thesis\Learning\GEE\Thuy
training_csv = os.path.join(out_dir, 'train_accuracy_rf.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy_rf.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

Reclassify land cover map

In [36]:
# landcover = landcover.remap(new_class_values, raw_class_values).select(['remapped'], ['classification'])

In [37]:
# landcover = landcover.set('classification_class_values', raw_class_values)
# landcover = landcover.set('classification_class_palette', class_palette)

In [38]:
# Map.addLayer(landcover, {}, 'Final land cover')
# Map

Export the result

In [39]:
import os
out_dir = os.path.join(os.path.expanduser('D:/Software/Jupyter/Thesis/Learning/GEE/Thuy/'), 'Results')
#D:\Software\Jupyter\Thesis\Learning\GEE\Thuy
out_file = os.path.join(out_dir, 'landcover_rf.tif')

In [40]:
# Export the result directly to your computer:
geemap.ee_export_image(classified, filename=out_file, scale=900)

Generating URL ...
Please wait ...
Data downloaded to D:\@MASTER THESIS\@Programming\@Jupyter\Learning\GEE\Thuy\Results\landcover_rf.tif


In [ ]:
Export the result to Google Drive:

In [ ]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)